# OPTUNA 

MIT licence.

3 principaux objectifs :
1. Construction de l'espace de recherche de façon dynamique (*define-by-run API*)
2. Implémentation efficiente de la recherche mais également du *pruning* (c'est-à-dire quand des branches entières de l'espace de recherche sont supprimées)
3. Utilisation simple allant de l'architecture légère à l'architecture extensible et distribuée.



To install : 

```shell
conda create -n optunenv python=3.9
conda install -c conda-forge optuna
conda install pandas
conda install pytorch torchvision torchaudio cpuonly -c pytorch
conda install -c anaconda scikit-learn
```

In [1]:
import pandas as pd
from pathlib import Path
from functools import partial
import numpy as np
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

import torch
import mytorch
import optuna

path_to_train_data = Path("data/Earthquakes_TRAIN.txt")
path_to_test_data = Path("data/Earthquakes_TEST.txt")

arr_train = np.loadtxt(path_to_train_data)
arr_test = np.loadtxt(path_to_test_data)
y_train = arr_train[:, 0].astype(int)
x_train = arr_train[:, 1:]
y_test = arr_test[:, 0].astype(int)
x_test = arr_test[:, 1:]

/home/paul/miniconda3/envs/optunenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
number_of_layers = [1, 2, 3, 4]
hidden_sizes = [1, 2, 3, 4]
lr_list = [1e-1, 1e-2, 1e-3, 1e-4]
batch_sizes = [4, 8, 16]

Let's start by defining the data, models, metrics used to illustrate OPTUNA.

In [3]:
model_class = partial(mytorch.TimeSeriesClassificationNet, input_size=1, number_of_classes=2, activation_fn='logsoftmax')

In [4]:
# model = model_class(hidden_size=hidden_sizes[0], num_layers=number_of_layers[0])
# mytorch.train(model, x_train, y_train, lr_list[0], epochs=10, batch_size=batch_sizes[0], lengths=[0.6, 0.4])

In [5]:
def objective_(trial, x, y):
    classifier_name = trial.suggest_categorical('classifier', ['SVR', 'RandomForest', 'GRU-classification-head'])
    if(classifier_name == 'SVR'):
        svr_c = trial.suggest_float('svr_c', 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVR(C=svr_c)
    elif(classifier_name  == 'RandomForest'):
        rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth)
    else:
        hidden_size = trial.suggest_int('hidden_size', 1, 8)
        num_layers = trial.suggest_int('num_layers', 1, 8)
        lr = trial.suggest_float('lr', 1e-5, 1, log=True)
        epochs = 10
        batch_size = 8
        model = model_class(hidden_size=hidden_size, num_layers=num_layers)
        classifier_obj = mytorch.TorchClassifier(model, lr, epochs, batch_size)

    x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x, y, random_state=0)
    classifier_obj.fit(x_train, y_train)
    y_pred = classifier_obj.predict(x_valid)

    error = sklearn.metrics.log_loss(y_valid, y_pred)
    return error

objective = partial(objective_, x=x_train, y=y_train)
study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100)  # Invoke optimization of the objective function.

[I 2023-01-28 19:24:07,662] A new study created in memory with name: no-name-1b1e865f-3767-4f1b-a8a7-e9706fac39ae
[W 2023-01-28 19:25:03,784] Trial 0 failed with parameters: {'classifier': 'GRU-classification-head', 'hidden_size': 5, 'num_layers': 7, 'lr': 0.00032370749956264647} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/paul/miniconda3/envs/optunenv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_136648/2050738999.py", line 19, in objective_
    classifier_obj.fit(x_train, y_train)
  File "/home/paul/Documents/repertoires/supaero-sdd-notebook-ml/mytorch.py", line 14, in fit
    train(self.model, x, y, self.lr, self.epochs, self.batch_size, self.device)
  File "/home/paul/Documents/repertoires/supaero-sdd-notebook-ml/mytorch.py", line 39, in train
    train_loss = train_one_epoch(epoch, model, optimiser, loss_function, train_loader)
  File

KeyboardInterrupt: 